# General Plotting Generator

In [10]:
from wrf import (to_np, getvar, interplevel)
from netCDF4 import Dataset

import os
from pathlib import Path

import numpy as np

import mayavi.mlab as mlab
import mayaviplot
mlab.init_notebook()

Notebook initialized with ipy backend.


# List of Cases

In [11]:
cases = []
cases += ["creek"]
cases += ["loyalton"]
cases += ["tor"]

### General Animation Transform

In [12]:
def root_transform(frame, max, root=2):
    return np.power(frame/max, 1/root)

### Animation Function Dictionary

In [13]:
aniFunc = {}

### General Smoke and Area

In [14]:

def smoke_and_area(file):
    nc = Dataset(file)
    hgt = to_np(getvar(nc, 'height_agl', units='m'))
    _, num_y, num_x = hgt.shape
    x3, y3, z3 = np.mgrid[0:num_x*50:50, 0:num_y*50:50, 50:4001:50]
    smoke = nc["fire_smoke"][:,:,:]
    smoke_interp = interplevel(smoke, hgt, 50)
    for h in range(100,4001,50):
        smoke_interp = np.dstack((smoke_interp, interplevel(smoke, hgt, h)))
    smoke_interp = smoke_interp.transpose(1, 0, 2)
    burn_area = (nc["FIRE_AREA"][0,0:4*num_y,0:4*num_x]).transpose(1, 0)
    x2, y2 = np.mgrid[0:num_x*50:50/4, 0:num_y*50:50/4]
    elevation= 0*x2
    return (x3, y3, z3, smoke_interp, x2, y2, elevation, burn_area)

### Creek Fire Case

##### Smoke Animation Settings

In [15]:
def animateCreekSmoke(frame, num_frames, azumith, elevation, distance, focalpoint):
    azumith -= 45*root_transform(frame, num_frames, 4)
    elevation += 60*root_transform(frame, num_frames, 4)
    distance += 3000*root_transform(frame, num_frames, 4)
    focalpoint[0] += 600*root_transform(frame, num_frames, 4)
    focalpoint[1] += 30*root_transform(frame, num_frames, 4)
    focalpoint[2] += 900*root_transform(frame, num_frames, 4)
    return (azumith, elevation, distance, focalpoint)

def creekViewSmoke(frame, num_frames):
    azumith = 315
    elevation = 30
    distance = 2000
    focalpoint = [5000, 2000, 0]
    a, e, d, f = animateCreekSmoke(frame, num_frames, azumith, elevation, distance, focalpoint)
    return (a, e, d, f)

aniFunc["creek"] = creekViewSmoke

### Loyalton Fire Case

##### Smoke Animation Settings

In [16]:
def animateLoyaltonSmoke(frame, num_frames, azumith, elevation, distance, focalpoint):
    azumith += 120*root_transform(frame, num_frames, 4)
    elevation += 60*root_transform(frame, num_frames, 4)
    distance += 5000*root_transform(frame, num_frames, 4)
    focalpoint[0] += 1200*root_transform(frame, num_frames, 4)
    focalpoint[1] += 1200*root_transform(frame, num_frames, 4)
    focalpoint[2] += 1000*root_transform(frame, num_frames, 4)
    return (azumith, elevation, distance, focalpoint)

def loyaltonViewSmoke(frame, num_frames):
    azumith = 225
    elevation = 30
    distance = 2000
    focalpoint = [2000, 2000, 0]
    a, e, d, f = animateLoyaltonSmoke(frame, num_frames, azumith, elevation, distance, focalpoint)
    return (a, e, d, f)

aniFunc["loyalton"] = loyaltonViewSmoke

### Idealized Tornado Vortex Case

##### Smoke Animation Settings

In [17]:
def animateTorSmoke(frame, num_frames, azumith, elevation, distance, focalpoint):
    azumith -= 70*root_transform(frame, num_frames, 5)
    elevation += 60*root_transform(frame, num_frames, 5)
    distance += 8000*root_transform(frame, num_frames, 5)
    focalpoint[0] += 1500*root_transform(frame, num_frames, 5)
    focalpoint[1] += 1500*root_transform(frame, num_frames, 5)
    focalpoint[2] += 1000*root_transform(frame, num_frames, 5)
    return (azumith, elevation, distance, focalpoint)

def torViewSmoke(frame, num_frames):
    azumith = 180
    elevation = 30
    distance = 2000
    focalpoint = [5000, 2000, 0]
    a, e, d, f = animateTorSmoke(frame, num_frames, azumith, elevation, distance, focalpoint)
    return (a, e, d, f)

aniFunc["tor"] = torViewSmoke

# Generate Smoke Plots

In [18]:
for case in cases:
    root = Path.cwd()/case
    files = os.listdir(root/"nc")
    for i in range(len(files)-1, -1, -1):
        if not files[i].startswith("wrfout"):
            files.pop(i)
    for frame, file in enumerate(files):
        x3, y3, z3, smoke_interp, x2, y2, elevation, burn_area = \
            smoke_and_area(root/"nc"/file)
        fig = mayaviplot.plot3d(x3, y3, z3, smoke_interp, x2, y2, elevation,
                                burn_area, aniFunc[case], frame, len(files)-1,
                                r_mv_cmap=True, squish=19.5, disp=0.5,
                                out_filename=str(root/"plots"/"smoke"/(str(frame) + ".png")))
        mlab.close(fig)
    print("Finished Plotting: {}".format(root))

(50, 205, 205)
(50, 205, 205)


KeyboardInterrupt: 